In [2]:
import cv2
import numpy as np

class EyeTracker:
    def __init__(self, face_cascade_path, eye_cascade_path):
        self.face_cascade = cv2.CascadeClassifier(face_cascade_path)
        self.eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

    def track(self, image):
        face_rects = self.face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        rects = []
        for (fx, fy, fw, fh) in face_rects:
            face_roi = image[fy:fy+fh, fx:fx+fw]
            eye_rects = self.eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20))
            for (ex, ey, ew, eh) in eye_rects:
                rects.append((fx+ex, fy+ey, fx+ex+ew, fy+ey+eh))
        return rects

def main():
    face_cascade_path = 'haarcascade_frontalface_default.xml'
    eye_cascade_path = 'haarcascade_eye.xml'
    et = EyeTracker(face_cascade_path, eye_cascade_path)

    camera = cv2.VideoCapture(0)

    while True:
        (grabbed, frame) = camera.read()
        if not grabbed:
            break
        frame = cv2.resize(frame, width=300)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = et.track(gray)
        for rect in rects:
            cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
        cv2.imshow('Tracking', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    camera.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()